<a href="https://colab.research.google.com/github/syeong1218/ysyy/blob/master/ch5_4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 5.4 단순한 계층 구현하기

곱셉노드를 'MulLayer', 덧셈노드를 'AddLayer'

### 5.4.1 곱셉계층

In [0]:
class MulLayer:
    def __init__(self): # 인스턴스 변수인 x,y를 초기화
        self.x = None
        self.y = None

    def forward(self, x, y): # x,y를 인수로 받고 두 값을 곱해서 반환
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [0]:
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward(순전파)
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("price:", int(price))

price: 220


In [0]:
# backward(역전파)
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dApple: 2.2
dApple_num: 110
dTax: 200


backward()가 받은 인수는 '순전파의 출력에 대한 미분'

mul_apple_layer은   순전파때는 apple_price를 출력, 역전파때는 apple_price의 미분 값인 dapple_price를 인수로 받는다.

### 5.4.2 덧셈 계층

In [0]:
class AddLayer:
    def __init__(self): #초기화가 필요없으니 pass
        pass

    def forward(self, x, y): # 입력받은 인수 x,y를 더해서 반환
        out = x + y

        return out

    def backward(self, dout): # 상류에서 내려온 미분을 그대로 하류로 보냄
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [0]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward(순전파)
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward(역전파)
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple_num:", int(dapple_num))
print("dApple:", dapple)
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple_num: 110
dApple: 2.2
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 5.5 활성화 함수 계층 구현하기

신경망을 구성하는 층 각각을 클래스 하나로 구현

### 5.5.1 ReLU 계층

* ReLU 수식
$$y=\left\{\begin{matrix}
x & (x>0) & \\ 
0 & (x\leq 0) & 
\end{matrix}\right.$$


$$\frac{\partial y}{\partial x}=\left\{\begin{matrix}
1 & (x>0) & \\ 
0 & (x\leq 0) & 
\end{matrix}\right.$$

순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보낸다. 하지만 x가 0 이하면 역전파는 하류로 신호를 보내지 않는다.

In [0]:
class Relu:
    def __init__(self):
        self.mask = None 
        # mask라는 변수를 갖는다. mask는 True/False로 구성된 넘파이 배열. 순전파의 입력인 x가 0 이하면 True, 0 이상이면 False

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [0]:
import numpy as np

x=np.array([[1.0, -0.5],[-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [0]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


### 5.5.2 Sigmoid 계층

* 시그모이드 함수
$$y=\frac{1}{1+exp(-x)}$$

  - **1단계**
  
      '/'노드
  $$y=\frac{1}{x}$$
  $$\frac{\partial y}{\partial x}=-\frac{1}{x^{2}}=-y^{2}$$
  
      역전파 때 상류에서 흘러온 값에 -y^2을 곱해서 하류로 전달.
      
  - **2단계**
  
     '+'노드는 상류의 값을 여과 없이 하류로 내보낸다.
     
  - **3단계**
  
     'exp'노드는 y=exp(x) 연산을 수행한다.
     $$\frac{\partial y}{\partial x}=exp(x)$$
     상류의 값에 순전파의 출력 exp(x)를 곱해 하류로 전파.
     
  - **4단계**
   
    'x'노드는 순전파 때의 값을 서로 바꿔 곱한다.
    
    
계산 그래프의 중간 과정을 모두 묶어 간소화 할 수 있다. 간소화 버전은 역전파 과정의 중간 계산들을 생략할 수 있어 더 효율적이고, 입력과 출력에만 집중 할 수 있다.

In [0]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6 Affine/Sortmax 계층 구현하기

### 5.6.1 Affine 계층
신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱을 사용.

In [0]:
X=np.random.rand(2)    # 입력
W=np.random.rand(2,3)  # 가중치
B=np.random.rand(3)    # 편향

X.shape

(2,)

In [0]:
W.shape

(2, 3)

In [0]:
B.shape

(3,)

In [0]:
Y=np.dot(X,W)+B

Y를 활성화함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름

**어파인 변환** : 신경망의 순전파 때 수행하는 행렬의 곱 ->어파인 변환을 수행하는 처리를 **Affine 계층**

#### 역전파

행렬의 역전파도 행렬의 원소마다 전개하면 스칼라값을 사용하여 계산하는 순서와 같게 생각 가능.

$$ \frac{\partial L}{\partial X}=\frac{\partial L}{\partial Y}\cdot W^{T}$$
$$ \frac{\partial L}{\partial W}=X^{T}\cdot \frac{\partial L}{\partial Y}$$

W^{T}의 T는 전치행렬(W의 (i,j)위치의 원소를 (j,i)위치로 바꾼 것을 말함.)을 의미.
$$ W=\begin{pmatrix}
w_{11} & w_{12} & w_{13}\\ 
w21 & w_{22} & w_{23}
\end{pmatrix}$$

$$W^{T}=\begin{pmatrix}
w_{11} & w_{12}\\ 
w_{12} & w_{22}\\ 
w_{13} & w_{23}
\end{pmatrix}$$

계산 그래프의 역전파 그림5-25

여기서 X와 dL/dX은 같은 형상, W와 dL/dW은 같은 형상

$$X=(x_{0},x_{1},\cdots ,x_{n})$$
$$\frac{\partial L}{\partial X}=(\frac{\partial L}{\partial x_{0}},\frac{\partial L}{\partial x_{1}},\cdots ,\frac{\partial L}{\partial x_{n}})$$

* 주의할 점:
    행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야한다.

### 배치용 Affine 계층

앞에서 설명한 것은 입력 데이터로 X 하나만 고려한 것.

여기서는 데이터 N 개를 묶어 순전파하는 경우 다룬다.

* N=2인 경우 예시

In [0]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [0]:
X_dot_W+B

array([[ 1,  2,  3],
       [11, 12, 13]])

순전파의 편향 덧셈은 각각의 데이터를 더한다.

그래서 역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모임.

In [0]:
dY = np.array([[1,2,3],[4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [0]:
dB=np.sum(dY, axis=0)
dB

array([5, 7, 9])

이 예는 데이터가 2개여서, 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구함. => np.sum()에서 0번째 축(데이터를 단위로 한 축)에 대해서 (axis=0)의 총합을 구하는 것.

In [0]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

이 데이터는 4차원 데이터를 고려한 것 

### 5.6.3 Softmax-with-Loss계층

소프트맥스 함수 : 입력 값을 정규화하여 출력

손글씨 숫자 인식을 예로 들면 숫자가 만약 2일 때 입력을 확률로 변환해 축력의 합이 1이 되도록 출력한다.


In [0]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

* 주의할 점 : 역전파 때는 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층으로 전파하는 점.

## 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림
* **전제** 
    
    신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 합니다. 신경망 학습은 다음과 같이 4단계로 수행합니다.


   **1단계 - 미니배치**

    훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표.
    
   **2단계 - 기울기 산출** -> 오차역전파법 등장
   
    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시.
    
   **3단계 - 매개변수 갱신**
    
    가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다.
    
   **4단계 - 반복**
    
    1~3단계를 반복합니다.

### 오차역전파법을 적용한 신경망 구현하기

In [0]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01): #초기화 수행
        # 가중치 초기화
        self.params = {} # params : 신경망의 매개변수를 보관
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #1번째층의 가중치
        self.params['b1'] = np.zeros(hidden_size)                                      #1번째층의 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)#2번째층의 가중치
        self.params['b2'] = np.zeros(output_size)                                      #2번째층의 편향

        # 계층 생성
        self.layers = OrderedDict() # layer : 신경망의 계층을 보관, OrderedDict : 순서가 있는 딕셔너리
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss() #lastlayer : 신경망의 마지막 계층
        
    def predict(self, x): #예측을 수행, 인수 x는 이미지 데이터
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t): #손실 함수의 값을 구한다
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t): # 정확도를 구한다
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t): #가중치 매개변수의 기울기를 수치미분방식으로 구한다.
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t): # 가중치 매개변수의 기울기를 오차역전파법으로 구한다.
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

### 오차역전파법으로 구한 기울기 검증하기
* 수치미분 : 느리다. 구현하기 쉽다.

* 오차역전파법 : 매개변수 많아도 효율적으로 계산 가능, 구현하기 복잡하다.

**기울기 확인: 수치미분과 오차역전파법으로 구한 기울기가 일치하는지 확인**

In [0]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.018733193775896e-10
b1:9.542260616119802e-10
W2:6.976710720632551e-08
b2:1.406114384344681e-07


* 수치미분과 오차역전파법의 결과 오차가 0이 되는일은 드물다. 컴퓨터가 할 수 있는 계산의 정밀도가 유한하기 때문이다. 따라서 올바르게 구현했다면 0에 아주 가까운 작은 값이 된다.

### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [0]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09863333333333334 0.0958
0.7884333333333333 0.7963
0.8756833333333334 0.8799
0.8997833333333334 0.9018
0.90875 0.9123
0.9168333333333333 0.9179
0.9206333333333333 0.9237
0.9254166666666667 0.9284
0.92925 0.9324
0.9322833333333334 0.9332
0.9349 0.9371
0.9375333333333333 0.9381
0.94 0.9408
0.9421166666666667 0.9416
0.9437833333333333 0.9418
0.9457 0.9449
0.94805 0.9457


* 4.5.3 과 비교
https://colab.research.google.com/github/syeong1218/ysyy/blob/master/ch_4_3_.ipynb